<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-project-v2/blob/main/BLM6114_project_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets pandas torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.3 MB/s eta 0:00:00


In [2]:
import pandas as pd

ds_question_types = pd.read_csv('gsm8k_tr_classified_merged.csv')
ds_question_types.head()

,question,answer,question_type,solution_method,question_type_old,solution_method_old,tr_llama_result,is_tr_llama_correct,gg_gemma_result,is_gg_gemma_correct
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,"{""type"": ""Oran orantı problemi"", ""explanation""...","{""method"": ""proportional calculation"", ""explan...",oran-orantı,yüzde hesaplama,Sonuç: 108000,0.0,Sonuç: 270 \n,0.0
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,"{""type"": ""Aritmetik Problem"", ""explanation"": ""...","{""method"": ""Aritmetik Hesaplama"", ""explanation...",yaş problemi,doğrudan çıkarma,Sonuç: 20,0.0,Sonuç: 21 \n,1.0
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,"{""type"": ""problem solving"", ""explanation"": ""Bu...","{""method"": ""beyin fırtınası"", ""explanation"": ""...",oran-orantı,birim oran,Sonuç: 8,0.0,Sonuç: 24 \n,1.0
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,"{""type"": ""Arithmetic Problem"", ""explanation"": ...","{""method"": ""Çarpma ve Toplama"", ""explanation"":...",çarpma,doğrudan çarpma,Sonuç: 360,0.0,Sonuç: 90 \n,0.0
4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...,"{""type"": ""Hız/Zaman Problemleri"", ""explanation...","{""method"": ""hesaplama"", ""explanation"": ""Kaplum...",oran-orantı,birim oran,Sonuç: 18,1.0,Sonuç: 18 \n,1.0


# Embedding işlemleri

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

df = pd.read_csv("gsm8k_tr_classified_merged.csv")
print("Veri seti okundu:")
print(df.head())

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-e5-large")
model     = AutoModel.from_pretrained("ytu-ce-cosmos/turkish-e5-large").to(device)

def embed_texts(texts, batch_size=16):
  all_embs = []
  for i in range(0, len(texts), batch_size):
    batch = texts[i : i+batch_size]
    enc = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
      last_hidden = model(**enc).last_hidden_state  # (B, L, D)
      emb = last_hidden.mean(dim=1)                 # mean pooling → (B, D)
    all_embs.append(emb.cpu())
  return torch.cat(all_embs, dim=0).numpy()

for col in ["question", "answer", "question_type", "solution_method"]:
  print(f"Embedding yapılıyor: {col}")
  embs = embed_texts(df[col].astype(str).tolist(), batch_size=16)
  df[f"{col}_emb"] = list(embs) # embedding'i sonuna "_emb" eklenerek yeni kolon olarak ekleniyor.
  print(f"Embedding tamamlandı: {col}")

df.to_csv("gsm8k_tr_with_embeddings.csv")


Veri seti okundu:
                                            question  \
0  Borris tekel bayisi her 6 ayda bir 90 kilogram...   
1  Mel, Katherine'den üç yaş küçük.  Katherine ik...   
2  James 2 ağacındaki tüm meyveleri toplar.  Her ...   
3  Kyle, her biri 15 origami yıldızı alabilen 2 c...   
4  Mark'ın iki evcil hayvanı var: Saniyede 10 adı...   

                                              answer  \
0  Borris şu anda her 6 ayda 90 kilogram üzüm kul...   
1  Katherine iki düzine yaşına geldiğinde 24 yaşı...   
2                           James 24 ağaç dikmiştir.   
3  Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...   
4  Kaplumbağanın yarışı berabere bitirmesi için 1...   

                                       question_type  \
0  {"type": "Oran orantı problemi", "explanation"...   
1  {"type": "Aritmetik Problem", "explanation": "...   
2  {"type": "problem solving", "explanation": "Bu...   
3  {"type": "Arithmetic Problem", "explanation": ...   
4  {"type": "Hız/Zaman Probl

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Embedding yapılıyor: question
Embedding tamamlandı: question
Embedding yapılıyor: answer
Embedding tamamlandı: answer
Embedding yapılıyor: question_type
Embedding tamamlandı: question_type
Embedding yapılıyor: solution_method
Embedding tamamlandı: solution_method


In [4]:
import pandas as pd

try:
    df_embeddings = pd.read_csv("gsm8k_tr_with_embeddings.csv")
    print("Embedding dosyası başarıyla yüklendi.")
    print("\nEmbedding içeren veri çerçevesinin ilk 5 satırı:")
    display(df_embeddings.head())
except FileNotFoundError:
    print("Hata: 'gsm8k_tr_with_embeddings.csv' dosyası bulunamadı.")
    print("Lütfen dosyanın doğru yolda olduğundan emin olun.")

Embedding dosyası başarıyla yüklendi.

Embedding içeren veri çerçevesinin ilk 5 satırı:


,Unnamed: 0,question,answer,question_type,solution_method,question_type_old,solution_method_old,tr_llama_result,is_tr_llama_correct,gg_gemma_result,is_gg_gemma_correct,question_emb,answer_emb,question_type_emb,solution_method_emb
0,0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,"{""type"": ""Oran orantı problemi"", ""explanation""...","{""method"": ""proportional calculation"", ""explan...",oran-orantı,yüzde hesaplama,Sonuç: 108000,0.0,Sonuç: 270 \n,0.0,[ 0.6715006 1.0025988 -0.8141864 ... -0.56...,[ 0.5446431 1.1449226 -0.8158145 ... 0.58...,[ 0.1897387 0.12631406 -0.6594379 ... 0.24...,[-0.08096628 0.57343423 -0.24044694 ... 0.42...
1,1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,"{""type"": ""Aritmetik Problem"", ""explanation"": ""...","{""method"": ""Aritmetik Hesaplama"", ""explanation...",yaş problemi,doğrudan çıkarma,Sonuç: 20,0.0,Sonuç: 21 \n,1.0,[ 0.52484596 -0.20660642 -0.56891537 ... 0.27...,[ 0.8434993 0.19339159 -0.5099972 ... 0.30...,[ 1.430924 -0.630825 -1.0841881 ... 0.31...,[ 1.091192 -0.30429524 -0.9432773 ... -0.06...
2,2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,"{""type"": ""problem solving"", ""explanation"": ""Bu...","{""method"": ""beyin fırtınası"", ""explanation"": ""...",oran-orantı,birim oran,Sonuç: 8,0.0,Sonuç: 24 \n,1.0,[ 0.10532897 -0.35827264 -1.2728502 ... -0.33...,[ 0.27943563 0.28232127 -0.38983282 ... -0.66...,[ 0.19272597 -0.4171816 -1.2453085 ... -0.08...,[ 0.91966903 -0.8279106 -0.5111354 ... -0.03...
3,3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,"{""type"": ""Arithmetic Problem"", ""explanation"": ...","{""method"": ""Çarpma ve Toplama"", ""explanation"":...",çarpma,doğrudan çarpma,Sonuç: 360,0.0,Sonuç: 90 \n,0.0,[-0.57047623 0.49581084 -0.5631299 ... 0.01...,[-0.59139633 0.22431299 -0.14734289 ... -0.17...,[ 0.324803 0.66421705 -0.27458137 ... 0.39...,[ 0.35767016 0.50170404 -0.4520758 ... 0.02...
4,4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...,"{""type"": ""Hız/Zaman Problemleri"", ""explanation...","{""method"": ""hesaplama"", ""explanation"": ""Kaplum...",oran-orantı,birim oran,Sonuç: 18,1.0,Sonuç: 18 \n,1.0,[ 0.30706674 0.04191275 -1.2288647 ... -0.75...,[-0.03708125 0.08095893 0.4905705 ... -0.35...,[ 0.49798334 -0.5233716 -0.4075574 ... 0.00...,[ 0.6446417 -0.42181966 -0.809587 ... -0.12...
